In [4]:
# --------------------------
# Single-cell Gradio Meeting Minutes App
# --------------------------

import os
import requests
import gradio as gr
from dotenv import load_dotenv

# --------------------------
# Load Hugging Face token
# --------------------------
load_dotenv()
HF_API_TOKEN = os.getenv("HF_API_TOKEN")

HEADERS = {
    "Authorization": f"Bearer {HF_API_TOKEN}"
}

WHISPER_URL = "https://api-inference.huggingface.co/models/openai/whisper-medium.en"
LLAMA_URL = "https://api-inference.huggingface.co/models/meta-llama/Llama-3.2-3B-Instruct"

# --------------------------
# Helper functions
# --------------------------

def extract_gdrive_file_id(url: str) -> str:
    """
    Extract file ID from Google Drive URL
    """
    return url.split("/d/")[1].split("/")[0]

def download_from_gdrive(file_id: str, output_path: str) -> str:
    """
    Download a public Google Drive file locally
    """
    download_url = "https://drive.google.com/uc?export=download"

    response = requests.get(
        download_url,
        params={"id": file_id},
        stream=True
    )
    response.raise_for_status()

    with open(output_path, "wb") as f:
        for chunk in response.iter_content(32768):
            if chunk:
                f.write(chunk)
    return output_path

def transcribe_audio(audio_path: str) -> str:
    """
    Transcribe audio using Whisper (HF API)
    """
    with open(audio_path, "rb") as audio:
        response = requests.post(
            WHISPER_URL,
            headers=HEADERS,
            data=audio
        )
    response.raise_for_status()
    return response.json()["text"]

def generate_minutes(prompt: str) -> str:
    """
    Generate meeting minutes using LLaMA
    """
    payload = {
        "inputs": prompt,
        "parameters": {
            "max_new_tokens": 2000,
            "temperature": 0.3
        }
    }

    response = requests.post(
        LLAMA_URL,
        headers=HEADERS,
        json=payload
    )
    response.raise_for_status()
    return response.json()[0]["generated_text"]

def build_prompt(transcription: str) -> str:
    SYSTEM_MESSAGE = """
You produce minutes of meetings from transcripts, with summary, key discussion points,
takeaways and action items with owners, in markdown format without code blocks.
"""
    return f"""
### Instruction:
{SYSTEM_MESSAGE}

### Transcript:
{transcription}

### Output:
"""

# --------------------------
# Main Gradio function
# --------------------------

def process_meeting(gdrive_url):
    """
    Google Drive URL -> Download -> Transcription -> Meeting Minutes
    """
    if not gdrive_url:
        return "❌ Please paste a Google Drive audio URL", ""

    try:
        file_id = extract_gdrive_file_id(gdrive_url)
    except Exception:
        return "❌ Invalid Google Drive URL", ""

    local_audio_path = "temp_meeting_audio.mp3"
    download_from_gdrive(file_id, local_audio_path)

    transcription = transcribe_audio(local_audio_path)
    prompt = build_prompt(transcription)
    minutes = generate_minutes(prompt)

    if os.path.exists(local_audio_path):
        os.remove(local_audio_path)

    return transcription, minutes

# --------------------------
# Launch Gradio UI
# --------------------------

with gr.Blocks() as demo:
    gr.Markdown("# 📝 AI Meeting Minutes Generator")
    gr.Markdown(
        "Paste a **public Google Drive audio link** below.\n\n"
        "Example:\n"
        "`https://drive.google.com/file/d/FILE_ID/view`"
    )

    gdrive_input = gr.Textbox(
        label="Google Drive Audio URL",
        placeholder="https://drive.google.com/file/d/XXXXXXXX/view"
    )

    transcription_output = gr.Markdown(label="Transcription")
    minutes_output = gr.Markdown(label="Meeting Minutes")

    generate_btn = gr.Button("Generate Minutes")
    generate_btn.click(
        fn=process_meeting,
        inputs=gdrive_input,
        outputs=[transcription_output, minutes_output]
    )

demo.launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "/Users/prasanthchamarthi/projects/llm_engineering/.venv/lib/python3.12/site-packages/gradio/queueing.py", line 759, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/prasanthchamarthi/projects/llm_engineering/.venv/lib/python3.12/site-packages/gradio/route_utils.py", line 354, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/prasanthchamarthi/projects/llm_engineering/.venv/lib/python3.12/site-packages/gradio/blocks.py", line 2116, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/prasanthchamarthi/projects/llm_engineering/.venv/lib/python3.12/site-packages/gradio/blocks.py", line 1623, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^